In [21]:
from collections import deque
import numpy as np
import math  

In [22]:

def isGoal(state, GOAL):
  return state == GOAL

def getZeroPosition(statte):
  for i in range(3):
    for j in range(3):
      if(statte[i][j]==0):
        return i,j 
  
  
def getNeighbours(state):
  neighbours = []

  i,j = getZeroPosition(state)
  
    
  if j+1<=2:
    right= np.copy(state).tolist()
    right[i][j]=right[i][j+1]
    right[i][j+1]=0
    neighbours.append(right)



  if i+1<=2:
    down=np.copy(state).tolist()
    down[i][j]= down[i+1][j]
    down[i+1][j]=0
    neighbours.append(down)



  if j-1>=0:
    left=np.copy(state).tolist()
    left[i][j]=left[i][j-1]
    left[i][j-1]=0
    neighbours.append(left)
    
  if i-1>=0:
    up=np.copy(state).tolist()
    up[i][j]=up[i-1][j]
    up[i-1][j]=0
    neighbours.append(up)

  return neighbours


def getPath(node):
    moves = []
    stack = deque()
    while node.parent is not None:
      stack.append(node)
      node = node.parent
    stack.append(node)
    
    print("number of moves: ",len(stack) - 1)
    while stack:
      moves.append(stack.pop().state)
    print(moves)
    return moves
      
def tupleToArray(tupple):
    return [list(item) for item in tupple]


def arrayToTuple(array):
    return tuple(map(tuple, array))


class Node(object):
    def __init__(self, parent, state):
      self.parent = parent
      self.state = state     

In [23]:
def DFS(state, GOAL):

  prevNode = Node(None, state)
  explored = set()
  frontier = deque()

  exploredCount = 0

  frontier.append(prevNode)

  while frontier:
      prevNode = frontier.pop()
      explored.add(arrayToTuple(prevNode.state))
      exploredCount = exploredCount + 1

      if(isGoal(prevNode.state, GOAL)):
        print("exploredCount: ",exploredCount)
        return prevNode

      neighbours = getNeighbours(prevNode.state)

      for neighbour in neighbours:
        if (not arrayToTuple(neighbour) in explored) and (not arrayToTuple(neighbour) in frontier):
          
          newNode = Node(prevNode, neighbour)
          frontier.append(newNode)

  return None

In [24]:
import queue 

def BFS(intial,goal):
  exploredNodes=0
  frontier = queue.Queue(maxsize=0) 
  currentNode = Node(None, intial)
  frontier.put(currentNode)
  explored=[]

  while not frontier.empty():
    currentNode = frontier.get()
    explored.append(currentNode.state)
    exploredNodes=exploredNodes+1
    
    state= currentNode.state
    if(currentNode.state==goal):
      print(exploredNodes,'total explored')
      return currentNode
    
    i,j= getZeroPosition(state)
    if i-1>=0:
      up=list(map(list, state))
      up[i][j]=up[i-1][j]
      up[i-1][j]=0
      if not up in explored:
        up= Node(currentNode, up)
        frontier.put(up)
    
    if j+1<=2:
      right= list(map(list, state))
      right[i][j]=right[i][j+1]
      right[i][j+1]=0
      if not right in explored:
        right= Node(currentNode, right)
        frontier.put(right)
        
    if i+1<=2:
      down=list(map(list, state))
      down[i][j]= down[i+1][j]
      down[i+1][j]=0
      if not down in explored:
        down = Node(currentNode, down)
        frontier.put(down)
        
        
    if j-1>=0:
      left=list(map(list, state))
      left[i][j]=left[i][j-1]
      left[i][j-1]=0
      if not left in explored:
        left= Node(currentNode, left)
        frontier.put(left)
  
  return None

In [25]:
def calculateCostManhattan(neighbour, GOAL, Gn):
  Hn = 0
  for i in range(3):
    for j in range(3):
      Hn = Hn + abs(i - neighbour[i][j] // 3) + abs(j - neighbour[i][j] - 3 * i)
  cost = Hn + Gn
#   print("Cost ",cost)
  return cost


def calculateCostEucledian(neighbour, GOAL, Gn):
  Hn = 0
  for i in range(3):
    for j in range(3):
      Hn = Hn + math.sqrt(pow(i - neighbour[i][j] // 3, 2) + pow(j - neighbour[i][j] - 3 * i, 2))
  cost = Hn + Gn
  return cost



def deleteFromHeap(neighbour, frontier):
  for idx, node in frontier:
    if neighbour == node:
      frontier[idx] = frontier[-1]
      frontier.pop()
      heapq.heapify(frontier)
      return frontier, node.cost

def printHeap(heap):
  for node in heap:
    print(node.cost,  end = '  ')

class NodeAStar(object):
    def __init__(self, parent, state, Gn, cost, depth):
      self.parent = parent
      self.state = state
      self.Gn = Gn
      self.cost = cost
      self.depth = depth

    def __eq__(self, other): 
      if not isinstance(other, NodeAStar):
        return self.cost == other 
      return self.cost == other.cost 
    
    def __lt__(self, other):
      if not isinstance(other, NodeAStar):
        return self.cost < other 
      return self.cost < other.cost
    
    def __le__(self, other):
      if not isinstance(other, NodeAStar):
        return self.cost <= other 
      return self.cost <= other.cost
    
    def __ne__(self, other):
      if not isinstance(other, NodeAStar):
        return self.cost != other 
      return not self.__eq__(other)
    
    def __gt__(self, other):
      if not isinstance(other, NodeAStar):
        return self.cost > other       
      return self.cost > other.cost
    
    def __ge__(self, other):
      if not isinstance(other, NodeAStar):
        return self.cost >= other       
      return self.cost >= other.cost
    
    def __hash__(self):
      return hash((self.parent, self.state, self.cost, self.Gn))

In [26]:
import heapq



def A_Star_Search(state, GOAL):
  frontier =[]
  explored = set()
  currentNode = NodeAStar(None, state,0, 0, 0)
  exploredCount = 0
  Gn = 0

  heapq.heappush(frontier,currentNode)


  
  while frontier:
#       printHeap(frontier)
      currentNode = heapq.heappop(frontier)
      explored.add(arrayToTuple(currentNode.state))
      exploredCount = exploredCount + 1

      if(isGoal(currentNode.state, GOAL)):
        print("ExploredCount A_Star: ", exploredCount)
        return currentNode

      neighbours = getNeighbours(currentNode.state)
      
      for neighbour in neighbours:
        if (not arrayToTuple(neighbour) in explored) and (not neighbour in frontier):
          
            cost = calculateCostEucledian(neighbour, GOAL, currentNode.Gn + 1)
          # cost = calculateCostManhattan(neighbour, GOAL, currentNode.Gn + 1)
            newNode = NodeAStar(currentNode, neighbour, currentNode.Gn + 1, cost, currentNode.depth + 1)
            heapq.heappush(frontier, newNode)
          
        elif neighbour in frontier:
            
            frontier, oldcost = deleteFromHeap(neighbour, frontier)
            print(oldcost)
            cost = calculateCostEucledian(neighbour, GOAL, currentNode.Gn + 1)
            print(cost)
            if(cost>oldcost):
                cost=oldcost
                
            newNode = NodeAStar(currentNode, neighbour, currentNode.Gn + 1, cost, currentNode.depth + 1)
            heapq.heappush(frontier, newNode)
            
            
          
  return None



In [27]:
import time

state = [[1,0,2],[7,5,4],[8,6,3]]
#state = [[1,2,5],[3,4,6],[0,7,8]]
GOAL = [[0,1,2],[3,4,5],[6,7,8]]

# state = [[1,2,3],[4,0,7],[5,6,8]]
# state = [[1,2,3],[4,5,7],[6,0,8]]
# state = [[1,4,2],[6,5,8],[7,3,0]]
# state = [[1,0,2],[7,5,4],[8,6,3]]

# newNode = DFS(state, GOAL)
# newNode = A_Star_Search(state, GOAL)
# states = getPath(newNode)
# startAnimation()

start_time = time.time()
newNode = A_Star_Search(state, GOAL)
states = getPath(newNode);
print("depth: ", newNode.depth)
print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [ ]:
!pip install pygame 
import pygame
import os

In [ ]:
def drawState(state):
    tempX = 75
    tempY = 75
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                text = font.render(str(state[i][j]), True, YELLOW, BLACK) 
                textRect = text.get_rect()  

                textRect.center = (tempX, tempY) 
                DISPLAY.blit(text, textRect)
                tempX=tempX + 150
            else:
                text = font.render('   ', True, BLACK, BLACK) 
                textRect = text.get_rect()

                textRect.center = (tempX, tempY) 
                DISPLAY.blit(text, textRect)
                tempX=tempX + 150
        tempX = 75
        tempY = tempY + 150
    pygame.display.update()
    time.sleep(1)
    
def endGame():
    text = font.render("Reached Goal", True, GREEN, BLACK) 
    textRect = text.get_rect()  

    textRect.center = (225, 225) 
    DISPLAY.blit(text, textRect)

In [ ]:
import sys, time

pygame.init()

DISPLAY=pygame.display.set_mode((450,450),0,32)
pygame.display.set_caption('Solving 8-Puzzle ----- Moves: 0') 


WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 153, 51)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
YELLOW = (255, 255, 0)


DISPLAY.fill(BLACK)

pygame.draw.line(DISPLAY,YELLOW,(150,0),(150,450),7)
pygame.draw.line(DISPLAY,YELLOW,(300,0),(300,450),7)
pygame.draw.line(DISPLAY,YELLOW,(0,150),(450,150),7)
pygame.draw.line(DISPLAY,YELLOW,(0,300),(450,300),7)

font = pygame.font.Font('freesansbold.ttf', 42) 

for idx, state in enumerate(states):      
    drawState(state)

    pygame.display.set_caption('Solving 8-Puzzle ----- Moves: ' + str(idx+1))
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit(0)
endGame()
pygame.display.update()


while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit(0)
    
